## CNNによる分類
torch.nn.Embeddingで単語をembeddingにした後CNNに入力 <br />
 - CNNの処理 <br />
  - 畳み込み：torch.nn.Conv2dを利用 <br />
  - プーリング：nn.functional.max_pool1dを利用 <br />

### データの読み込みと整形
単語分割 -> 各単語をidに変換

In [3]:
from data import Corpus # 自作クラス
dataset = Corpus("text_data.txt")
vocab_size = len(dataset.word_to_id_dict) # 総語彙数 (入力サイズ)
output_size = len(dataset.label_to_id_dict) # ラベル数 (出力サイズ)

### CNNによるミニバッチ処理のための前処理
ミニバッチ処理ではすべてのデータを同様に処理したいので，<br />
系列長を合わせるためにゼロ埋め（パディング）する．<br />

In [25]:
import torch
from torch import nn

pad_sentence_list = nn.utils.rnn.pad_sequence([torch.tensor(sentence.word_id_list) for sentence in dataset.train_sentence_list], batch_first=True)
pad_sentence_list

tensor([[ 4,  5,  6,  7,  8,  0,  0,  0],
        [ 9, 10, 11, 12, 13, 14, 15, 16],
        [17, 18, 10, 19, 20,  6,  0,  0],
        [21, 22, 23, 10, 19,  0,  0,  0],
        [24, 25, 26, 27, 28, 29,  0,  0],
        [30, 10, 14, 31, 32,  0,  0,  0]])

### Embedding層の定義

In [26]:
embedding_size = 5
embeddings = nn.Embedding(vocab_size, embedding_size, padding_idx=0)

### Embedding層への入力

In [27]:
sentence_embeds = embeddings(pad_sentence_list) 

### CNN入力のためのEmbeddingのreshape

In [28]:
print("reshape前", sentence_embeds.size())
sentence_embeds = sentence_embeds.unsqueeze(1)
print("reshape後", sentence_embeds.size())

reshape前 torch.Size([6, 8, 5])
reshape後 torch.Size([6, 1, 8, 5])


### CNNへの入力
#### 畳み込み: nn.Conv2dを利用

In [33]:
# nn.Conv2dの定義
kernel_size = 3
out_channels = 5
convs = nn.Conv2d(1, out_channels, (kernel_size, embedding_size))
print("畳み込みのための重みテンソルのサイズ", convs.weight.size())
print("畳み込みのためのバイアステンソルのサイズ", convs.bias.size())

畳み込みのための重みテンソルのサイズ torch.Size([5, 1, 3, 5])
畳み込みのためのバイアステンソルのサイズ torch.Size([5])


<br />
<img src="figures/conv2d.jpg" width="720px" align="left"><br clear="all" />
<br />

In [52]:
conv_outputs = convs(sentence_embeds)
print(conv_outputs.size(), "\n=> [バッチサイズ, out_channels, kernelによって切り取られる数, 1]")
print("単語数 : {} でkernel_size : {} のとき切り取られる数 : {}".format(sentence_embeds.size(2), kernel_size, sentence_embeds.size(2)-kernel_size+1))

torch.Size([6, 5, 6, 1]) 
=> [バッチサイズ, out_channels, kernelによって切り取られる数, 1]
単語数 : 8 でkernel_size : 3 のとき切り取られる数 : 6


In [58]:
# 結果に活性化関数を適用
tanh_conv_outputs = torch.tanh(conv_outputs)
print(tanh_conv_outputs.size())

torch.Size([6, 5, 6, 1])


#### プーリング: nn.functional.max_pool1dを利用

In [59]:
tanh_conv_outputs = tanh_conv_outputs.squeeze(3) # pool1d入力のための整形
print(tanh_conv_outputs.size())
pooled = nn.functional.max_pool1d(tanh_conv_outputs, tanh_conv_outputs.size(2)).squeeze(2)

torch.Size([6, 5, 6])


#### dropoutの適用と出力層への入力

In [63]:
# dropout層の定義
dropout = nn.Dropout()
# 出力層の定義
output_linear = nn.Linear(1*out_channels, output_size)

In [67]:
pooled_dropout = dropout(pooled)
output = output_linear(pooled_dropout)
print(output) # 6データ（1バッチ）の出力 

tensor([[ 0.3329, -0.3019],
        [ 0.4947, -0.5262],
        [ 0.5466, -0.5484],
        [ 0.3540, -0.2262],
        [ 0.4564, -0.3188],
        [ 0.0210, -0.1269]], grad_fn=<AddmmBackward>)
